In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense

import seaborn as sns
import re
import datetime
import os


In [ ]:
# %tensorflow_version 2.x
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
import glob
drive.mount('/content/drive')

Mounted at /content/drive


# CICIDS 2018 합치는 코드


원래 쓰던 csv 파일 하나에 Infi? 어쩌구 공격 하나만 들어있어서
일단 DDos, Dos 계열 공격 들어있는 파일 세개 합침
(전체 파일 다 합치려고 하니까 램 딸려서 안됨)

In [ ]:
files = [file for file in glob.glob("/content/drive/MyDrive/temp/*.csv", recursive=True)]
for f in files:
  print(f)

/content/drive/MyDrive/temp/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/MyDrive/temp/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/MyDrive/temp/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/MyDrive/temp/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv


In [ ]:
dataset = [pd.read_csv(f) for f in files]
for d in dataset:
    print(d.shape)
# 모든 DataFrame이 같은 열을 갖고 있는지 체크
# for i in range(0,len(dataset)):
#     if i != len(dataset)-1:
#         same_columns = dataset[i].columns == dataset[i+1].columns
        
#         if False in same_columns:
#             print(i)
#             break

(592035, 79)
(289599, 79)
(876946, 79)
(561405, 79)


In [ ]:
dataset = pd.concat([d for d in dataset]).drop_duplicates(keep=False)

# 전처리 과정에서 인덱스 뒤죽박죽됨
# 기존 인덱스 값 제거하고(drop=True), 정수 인덱스 세팅 
dataset.reset_index(drop=True, inplace = True)

In [ ]:
dataset.shape

(2318022, 79)

# CICIDS 2018 Test

In [ ]:
# dataset = pd.read_csv("/content/drive/MyDrive/CICIDS2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv")

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318022 entries, 0 to 2318021
Data columns (total 79 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Flow Duration      int64  
 3   Tot Fwd Pkts       int64  
 4   Tot Bwd Pkts       int64  
 5   TotLen Fwd Pkts    int64  
 6   TotLen Bwd Pkts    int64  
 7   Fwd Pkt Len Max    int64  
 8   Fwd Pkt Len Min    int64  
 9   Fwd Pkt Len Mean   float64
 10  Fwd Pkt Len Std    float64
 11  Bwd Pkt Len Max    int64  
 12  Bwd Pkt Len Min    int64  
 13  Bwd Pkt Len Mean   float64
 14  Bwd Pkt Len Std    float64
 15  Flow Byts/s        float64
 16  Flow Pkts/s        float64
 17  Flow IAT Mean      float64
 18  Flow IAT Std       float64
 19  Flow IAT Max       int64  
 20  Flow IAT Min       int64  
 21  Fwd IAT Tot        int64  
 22  Fwd IAT Mean       float64
 23  Fwd IAT Std        float64
 24  Fwd IAT Max        int64  
 25  Fwd IAT Min       

In [ ]:
dataset['Label'].unique()

array(['Benign', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk',
       'Infilteration', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DDOS attack-LOIC-UDP', 'DDOS attack-HOIC'], dtype=object)

In [ ]:
attack_class = {
'Benign' : 'BENIGN',

'DDOS attack-LOIC-UDP' : 'DDOS',
'DDOS attack-HOIC' : 'DDOS',

'DoS attacks-GoldenEye' : 'DoS',
'DoS attacks-Hulk' : 'DoS',
'DoS attacks-SlowHTTPTest' : 'DoS',
'DoS attacks-Slowloris' : 'DoS',
'FTPPatator' : 'Brute Force',
'SSH-Bruteforce': 'Brute Force',
'Web_Attack_Brute_Force' : 'Brute Force',
'FTP-BruteForce': 'Brute Force',
'Brute Force -Web' : 'Brute Force',
'Brute Force -XSS' : 'Brute Force',
'Web_Attack_Sql_Injection' : 'Web Attack',
'SQL Injection' : 'Web Attack',
'Web_Attack_XSS' : 'Web Attack',
'Bot' : 'Botnet',
'PortScan' : 'Port Scan',
'Heartbleed' : 'Web Attack',
'SSHPatator' : 'Web Attack'
}
dataset = dataset.replace({'Label' : attack_class})
np.unique(dataset.Label)

array(['BENIGN', 'DDOS', 'DoS', 'Infilteration'], dtype=object)

In [ ]:
change_label = {
    'BENIGN': 0,
    'Botnet': 1, 
    'Brute Force': 2,
    'DDOS' : 3,
    'DoS' : 4,
    'Infilteration': 5,
    'Port Scan': 6,
    'Web Attack' : 7
}
dataset = dataset.replace({'Label' : change_label})
np.unique(dataset['Label'])

array([0, 3, 4, 5])

In [ ]:
#필요없는 행 제거
dataset = dataset.drop(["Protocol"],axis = 1)
# dataset = dataset.drop(["Timestamp"],axis = 1)

In [ ]:
# Column 이름 공백 제거
col_names = [col.replace(' ', '') for col in dataset.columns]
dataset.columns = col_names

# Label 값 다듬기
# label_names = dataset['Label'].unique()
# label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names] 
# label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
# label_names = [lab.replace("__", "_") for lab in label_names]

# labels = dataset['Label'].unique()
# for i in range(0,len(label_names)):
#     dataset['Label'] = dataset['Label'].replace({labels[i] : label_names[i]})

# 결측치 제거
dataset.dropna(inplace=True)

# 무한대인 것 제거
labl = dataset['Label']
dataset = dataset.apply(pd.to_numeric, errors='coerce')
dataset = dataset.loc[:, dataset.columns != 'Label'].astype('float64')
dataset = dataset.replace([np.inf, -np.inf], np.nan)
dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)
dataset.dropna(inplace=True)




In [ ]:
np.unique(dataset['Label'])

array([0, 3, 4, 5])

In [ ]:
# 데이터 스케일링
labels = dataset['Label']
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

scaler = MinMaxScaler() # 한번 바꿔봄..
scaler.fit(features)
features = scaler.transform(features)

In [ ]:
np.unique(labels)

array([0, 3, 4, 5])

In [ ]:
model_path = '/content/drive/MyDrive/AEwithLabel_sy.h5'
autoencoder = load_model (model_path)

In [ ]:
predictions = autoencoder.predict(features)

In [ ]:
predictions

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
idx = np.argmax(predictions, axis = 1)

In [ ]:
idx = pd.DataFrame(idx, columns = ['Label'])

In [ ]:
# change_label = {
#           0: 'Benign', 
#           1: 'Bot', 
#           2: 'DDoS', 
#           3: 'DoS_GoldenEye', 
#           4: 'DoS_Hulk',
#           5: 'DoS_Slowhttptest', 
#           6: 'DoS_slowloris', 
#           7: 'FTPPatator', 
#           8: 'Heartbleed',
#           9: 'Infilteration', 
#           10: 'PortScan', 
#           11: 'SSHPatator', 
#           12: 'Web_Attack_Brute_Force',
#           13: 'Web_Attack_Sql_Injection', 
#           14: 'Web_Attack_XSS'
# }


# change_label = {
#     0: 'BENIGN',
#     1: 'Botnet',
#     2: 'Brute Force',
#     3: 'DDoS',
#     4: 'DoS',
#     5: 'Infiltration',
#     6: 'Port Scan',
#     7: 'Web Attack'
# }

In [ ]:
predicted_label = idx['Label']

In [ ]:
# predicted_label = idx.replace({'Label' : change_label})

In [ ]:
np.unique(predicted_label)

array([0, 2, 3, 4, 6])

In [ ]:
np.unique(labels)

array([0, 3, 4, 5])

In [ ]:
predicted_label = predicted_label.tolist()

In [ ]:
labels = labels.tolist()

In [ ]:
from sklearn.metrics import classification_report

# target_names = ['BENIGN', 'Botnet', 'Brute Force', 'DDoS', 'DoS', 'Infiltration','Port Scan', 'Web Attack']

In [ ]:
report = classification_report(labels, predicted_label)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
print(report) # 여기서 나온 weighted avg 갖다 쓰면 되려나... 

              precision    recall  f1-score   support

           0       0.80      1.00      0.89   1838349
           2       0.00      0.00      0.00         0
           3       0.92      0.01      0.02    200591
           4       0.93      0.02      0.05    196551
           5       0.00      0.00      0.00     82531
           6       0.00      0.00      0.00         0

    accuracy                           0.79   2318022
   macro avg       0.44      0.17      0.16   2318022
weighted avg       0.79      0.79      0.71   2318022



In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_auc_score, roc_curve

# accuracy = accuracy_score(labels, predicted_label)
# precision = precision_score(labels, predicted_label,average='micro')
# recall = recall_score(labels, predicted_label,average='micro')
# f1 = f1_score(labels, predicted_label,average='micro')
# accuracy, precision, recall, f1